# Sensitivity analysis of the NF-kB pathway
The steps for this are as follow:
1. From an equations text file, load the reaction equations and create a state space model
2. From the state space model and initial parameters, find 
    1. The initial parameters for the model (run it 1000 times from the first initial conditions)
    2. Determine the value of x<sub>i</sub> for all i for all t in [0, N]
3. Calculate s<sub>i, j</sub> for all t given the values of x<sub>i</sub> for all t
4. Calcuate sbar for all t
5. Having the sensitivity matrix and the values of x for all desired t, calculate the sensitivity equations

We need x<sub>i</sub>(t) and s<sub>i, j</sub> to calculate sbar, which is needed for sensitivity analysis. Sensitivity analysis for all constants is defined as 
![Equation 22](./equation22.png)
where s bar is
![s bar](./sbar.png)
dx/dtheta can be found from S
![S](./bigS.png)
and an entry  s<sub>i, j</sub> is 
![s](./littleS.png)
where x<sub>i</sub> is the integration of the system of equations from the state space model and delta is the Kronecker delta function 
![kronecker delta](./kroneckerDelta.png)
### All necessary imports

In [1]:
import copy
import numpy as np
import sympy as sp
import math
from scipy.integrate import solve_ivp

### All program constants

In [2]:
initial_conditions_N = 2000
N = 400

pathway_file = 'pathway_eqs.txt'
reaction_coefficients_file = 'reaction_coefficients.txt'

### Load the constants and separate them 

In [3]:
ks = {}
with open(reaction_coefficients_file, 'r') as o: 
    for l in o:
        k = l.split('=')[0]
        ks[k] = float(eval(l.split('=')[1].replace('\n', '').strip()))

## 1. Generate the state space model from reaction equations

In [4]:
import numpy as np

file = open(pathway_file,"r")
eqs_arr=file.readlines()
file.close()
eqs_arr=[i[:-1] if i[-1] == '\n' else i for i in eqs_arr ] #remove \n

class Rxn:
    
    def __init__(self, ID, reactants, products):
        #reactants and products are lists
        self.ID = str(ID)
        self.reactants = reactants
        self.products = products
        
    def __str__(self):
        ret = "ID: " + self.ID + " Reactants: "
        for r in self.reactants:
            ret = ret + r.ID + ", "
        ret = ret + "Products: "
        for p in self.products:
            ret = ret + p.ID + ", "
        return ret

class RxnSpec:
    
    def __init__(self, ID, name):
        self.ID = 'x' + str(ID)
        self.name = name
        
    def __str__(self):
        return 'ID: ' + self.ID + ' name: ' + self.name

#list of rxn species
rxn_species = [RxnSpec(1,'ikba'), RxnSpec(2,'nf-kb'), RxnSpec(3,'ikba-nf-kb'), RxnSpec(4,'ikbb'), RxnSpec(5,'ikbb-nf-kb'),
              RxnSpec(6,'ikbe'),RxnSpec(7,'ikbe-nf-kb'),RxnSpec(8,'ikkikba'),RxnSpec(9,'ikkikba-nf-kb'),RxnSpec(10,'ikk'),
              RxnSpec(11,'ikkikbb'),RxnSpec(12,'ikkikbb-nf-kb'),RxnSpec(13,'ikkikbe'),RxnSpec(14,'ikkikbe-nf-kb'),
              RxnSpec(15,'nf-kb_n'),RxnSpec(16,'ikba_n'),RxnSpec(17,'ikba_n-nf-kb_n'),RxnSpec(18,'ikbb_n'),RxnSpec(19,'ikbb_n-nf-kb_n'),
              RxnSpec(20,'ikbe_n'),RxnSpec(21,'ikbe_n-nf-kb_n'),RxnSpec(22,'ikba_-t'),RxnSpec(23,'ikbb_-t'),RxnSpec(24,'ikbe_-t'),RxnSpec('SOURCE','source'),RxnSpec('SINK','sink')]

rxn_arr = []

for idx,eq in enumerate(eqs_arr):
    temp = eq.split('->')
    reactants = temp[0].split('+')
    products = temp[1].split('+')
    #convert each element in reactants and products into a RxnSpec
    new_reactants = []
    new_products = []
    for reac in reactants:
        for rs in rxn_species:
            if rs.name == reac:
                new_reactants.append(rs)
                break
    for prod in products:
        for rs in rxn_species:
            
            if rs.name == prod:
                new_products.append(rs)
                break
    
    
    rxn = Rxn(idx+1,new_reactants, new_products)
    rxn_arr.append(rxn)

state_space_model = []

for rs in rxn_species:
    eq = rs.ID + '(t) = '
    for rxn in rxn_arr:
        if rs.name in [i.name for i in rxn.reactants]:
            k = '-k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
            
        if rs.name in [i.name for i in rxn.products]:
            k = '+k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
    state_space_model.append(eq)
    
for eq in state_space_model:
    print(eq)

x1(t) = -k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
x2(t) = -k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15
x3(t) = +k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17
x4(t) = -k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18
x5(t) = +k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19
x6(t) = -k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20
x7(t) = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
x8(t) = -k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8
x9(t) = +k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9
x10(t) = +k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13
x11(t) = -k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11
x12(t) = +k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12
x13(t) = -k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13
x14(

## 2. Identify the initial conditions and x(t) for all t
1. Run the integration initial_conditions_N number of times to get initial conditions
2. Augment these by any thing you need from the paper for actual initial conditions

__NOTE__:   as of now, hard code the k values (from some source) and the partial derivatives (from the state space model)

In [5]:
# followed the direction of this video https://www.youtube.com/watch?v=zRMmiBMjP9o
def nf_kb(t, x):
    # x is here to be used by (1) initial conditions and (2) x when we use odeint
    # t is here for obvious reasons
    
    # define the k values
    k1=0.5
    k2=0.5*10**-3 
    k3=0.5
    k4=0.5*10**-3 
    k5=0.5
    k6=0.5*10**-3 
    k7=0.5
    k8=0.5*10**-3 
    k9=2.04*10**-2 
    k10=0.5
    k11=0.5*10**-3 
    k12=7.5*10**-3 
    k13=0.5
    k14=0.5*10**-3 
    k15=1.1*10**-2 
    k16=2.25*10**-5 
    k17=2.25*10**-5 
    k18=2.25*10**-5 
    k19=0.9*10**-1 
    k20=0.8*10**-4 
    k21=0.5
    k22=0.5*10**-3 
    k23=0.5
    k24=0.5*10**-3 
    k25=0.5
    k26=0.5*10**-3 
    k27=1.54*10**-6  
    k28=1.65*10**-2  
    k29=2.8*10**-4 
    k30=1.78*10**-7  
    k31=2.8*10**-4 
    k32=1.27*10**-7  
    k33=2.8*10**-4 
    k34=22.5*10**-3  
    k35=1.25*10**-3 
    k36=4.08*10**-3 
    k37=1.13*10**-4 
    k38=3*10**-4 
    k39=2*10**-4 
    k40=6.0*10**-3  
    k41=1.75*10**-3 
    k42=4.08*10**-3 
    k43=1.13*10**-4 
    k44=1.5*10**-4 
    k45=1*10**-4 
    k46=9.0*10**-3  
    k47=1.75*10**-3 
    k48=4.08*10**-3 
    k49=1.13*10**-4 
    k50=1.5*10**-4 
    k51=1*10**-4 
    k52=1.85*10**-1  
    k53=1.25*10**-3 
    k54=1.38*10**-2 
    k55=4.8*10**-2  
    k56=1.75*10**-3 
    k57=5.2*10**-3 
    k58=7.0*10**-2  
    k59=1.75*10**-3 
    k60=5.2*10**-3 
    k61=1.2*10**-4 
    k62=4.07*10**-3 
    k63=1.5*10**-3 
    k64=2.2*10**-3 
    
    # im lazy and make it easy to read
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]
    x5 = x[4]
    x6 = x[5]
    x7 = x[6]
    x8 = x[7]
    x9 = x[8]
    x10 = x[9]
    x11 = x[10]
    x12 = x[11]
    x13 = x[12]
    x14 = x[13]
    x15 = x[14]
    x16 = x[15]
    x17 = x[16]
    x18 = x[17]
    x19 = x[18]
    x20 = x[19]
    x21 = x[20]
    x22 = x[21]
    x23 = x[22]
    x24 = x[23]
    
    dx1dt = -k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
    dx2dt = -k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15
    dx3dt = +k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17
    dx4dt = -k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18
    dx5dt = +k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19
    dx6dt = -k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20
    dx7dt = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
    dx8dt = -k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8
    dx9dt = +k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9
    dx10dt = +k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13
    dx11dt = -k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11
    dx12dt = +k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12
    dx13dt = -k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13
    dx14dt = +k13*x13*x2-k14*x14-k15*x14+k58*x10*x7-k59*x14
    dx15dt = +k19*x2-k20*x15-k21*x16*x15+k22*x17-k23*x18*x15+k24*x19-k25*x20*x15+k26*x21-k28*x15*x15+k28*x15*x15
    dx16dt = -k21*x16*x15+k22*x17+k38*x1-k39*x16
    dx17dt = +k21*x16*x15-k22*x17-k54*x17
    dx18dt = -k23*x18*x15+k24*x19+k44*x4-k45*x18
    dx19dt = +k23*x18*x15-k24*x19-k57*x19
    dx20dt = -k25*x20*x15+k26*x21+k50*x6-k51*x20
    dx21dt = +k25*x20*x15-k26*x21-k60*x21
    dx22dt = +k27+k28*x15*x15-k29*x22-k36*x22+k36*x22
    dx23dt = +k30-k31*x23-k42*x23+k42*x23
    dx24dt = +k32-k33*x24-k48*x24+k48*x24
    
    
    return [dx1dt, dx2dt, dx3dt, dx4dt, dx5dt, dx6dt, dx7dt, dx8dt,
            dx9dt, dx10dt, dx11dt, dx12dt, dx13dt, dx14dt, dx15dt, dx16dt, 
            dx17dt, dx18dt, dx19dt, dx20dt, dx21dt, dx22dt, dx23dt, dx24dt]
# first run for the initial conditions
T = initial_conditions_N 
x0x0 = [0, 0.1] + [0 for i in range(22)]
t = [i for i in range(T+1)]
init_conditions = solve_ivp(nf_kb, (min(t), max(t)), x0x0, t_eval=t).y[:, -1] # only want the last values


Now we have the initial conditions, so we need to augment this by paper specific parameters

In [7]:
# ok now do it for the simulation
T = N + 1
x0 = init_conditions
# fill in the extra stuff in the paper, IKK at 0.1 (x10, index 9) PAPER SPECIFIC
x0[9] = 0.1
t = [i for i in range(T)]

x = solve_ivp(nf_kb, (min(t), max(t)), x0, t_eval=t).y
# x has the shape of 
# [N, i] (an entry for all x for all N)
# replace x[0] with the initial conditions
x = x[:, 1:]


(24, 400)


## 3. Calculate s for all t
### Finding dx with respect to theta
Here are the equations

![dx dtheat](dxdtheta.png)

Once we can get equation 9, we can integrate with respect to time to get dx dtheta.

s i j is defined as 

![s](littleS.png)

I think f are the system of equations from the state space model

I'm using the variable `p` and `p_i` to fill in for dx/dtheta

In [8]:
# here is the state space model

ssm = []
ssm.append(sp.sympify('-k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16'))
ssm.append(sp.sympify('-k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15'))
ssm.append(sp.sympify('+k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17'))
ssm.append(sp.sympify('-k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18'))
ssm.append(sp.sympify('+k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19'))
ssm.append(sp.sympify('-k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20'))
ssm.append(sp.sympify('+k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21'))
ssm.append(sp.sympify('-k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8'))
ssm.append(sp.sympify('+k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9'))
ssm.append(sp.sympify('+k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13'))
ssm.append(sp.sympify('-k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11'))
ssm.append(sp.sympify('+k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12'))
ssm.append(sp.sympify('-k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13'))
ssm.append(sp.sympify('+k13*x13*x2-k14*x14-k15*x14+k58*x10*x7-k59*x14'))
ssm.append(sp.sympify('+k19*x2-k20*x15-k21*x16*x15+k22*x17-k23*x18*x15+k24*x19-k25*x20*x15+k26*x21-k28*x15*x15+k28*x15*x15'))
ssm.append(sp.sympify('-k21*x16*x15+k22*x17+k38*x1-k39*x16'))
ssm.append(sp.sympify('+k21*x16*x15-k22*x17-k54*x17'))
ssm.append(sp.sympify('-k23*x18*x15+k24*x19+k44*x4-k45*x18'))
ssm.append(sp.sympify('+k23*x18*x15-k24*x19-k57*x19'))
ssm.append(sp.sympify('-k25*x20*x15+k26*x21+k50*x6-k51*x20'))
ssm.append(sp.sympify('+k25*x20*x15-k26*x21-k60*x21'))
ssm.append(sp.sympify('+k27+k28*x15*x15-k29*x22-k36*x22+k36*x22'))
ssm.append(sp.sympify('+k30-k31*x23-k42*x23+k42*x23'))
ssm.append(sp.sympify('+k32-k33*x24-k48*x24+k48*x24'))

In [9]:
# get Sj. ONLY FOR INITIAL CONDITIONS OF SOLVING THE NEXT SYSTEM OF EQUATIONS
def kdf(i, j):
    if i != j:
        return 0
    return ks['k{}'.format(j+1)] - x0[i]

def sj(j):
    return [kdf(i, j) for i in range(len(x0))]

In [10]:
# get Fj (df/dthetaj)
def fj(j):
    return [sp.diff(ssm[i], 'k{}'.format(j+1)) for i in range(len(x0))]

In [11]:
# find J
def makeJ():
    J = []
    # df1 is top row where as dx1 is the first column
    # is dfn/dfn
    for i in range(len(ssm)):
        row = []
        tod = ssm[i]
        for n in range(len(ssm)):
            respectto = 'x{}'.format(n+1)
            row.append(sp.diff(tod, respectto))
        J.append(row)
    return J

In [12]:
J = makeJ()
def ddtdxdthetaj(j, p):
    return np.add(np.dot(np.asarray(J), p), np.asarray(fj(j)))

In [13]:
# linear interpolation for x 
# x is i by N shape, so index i then t
thresh = .000001
def interpx(i, t):
    if abs(t - int(t)) < thresh:
        return x[i][int(t)]
    t1 = int(math.floor(t))
    t2 = int(math.ceil(t))
    x1 = x[i][t1]
    x2 = x[i][t2]
    return ((t-t1) * (x2 - x1)/(t2-t1)) + x1

In [40]:
def integratethis(t, p, soe):
    tosub = {}
    tosolve = copy.deepcopy(soe)
    for i in range(len(x0)):
        tosub['x{}'.format(i + 1)] = interpx(i, t)
    for k in ks:
        tosub[k] = ks[k]
    for pi in range(len(p)):
        tosub['p{}'.format(pi+1)] = p[pi]
    for j in range(len(tosolve)):
        tosolve[j] = float(tosolve[j].evalf(subs=tosub))
    return list(tosolve)

N = 400
t = [i for i in range(N)]
# the form of this solved system will be (j, N, i) so first index is the theta, the second is the time, and the third is the x
solveddxdtheta = []
for j in range(len(ks)): 
    print('solving dx dtheta for j {}/{}\r'.format(j+1, len(ks)), end='')
    soe = ddtdxdthetaj(j, [sp.sympify('p{}'.format(i + 1)) for i in range(len(x0))])
    p0 = sj(j)
    solveddxdtheta.append(solve_ivp(integratethis, (min(t), max(t)), p0, args=(soe, ), t_eval=t).y)
    

KeyboardInterrupt: 

## 4. S bar
s bar is defined as:
![s bar](./sbar.png)

In [36]:
# find sbar for all t
def sbar(i, j, k):
    return solveddxdtheta[j][k][i] * ks['k{}'.format(j+1)] / x[i][k]

## 5 the sensitivity constants 

### Dynamic sensitivity analysis

#### Dynamic sensitivity analysis with multiple variables
This is equation 22 of the paper
![Equation 22](./equation22.png)

In [37]:
# do the innermost summation
import math
def inner_sum(k, j, i_range):
    tosum = []
    for i in range(i_range):
        added = sbar(i, j, k)
        tosum.append(sbar(i, j, k) ** 2)
    return sum(tosum)

In [38]:
# do the outer summation
def outer_sum(N, j, i_range):
    tosum = []
    for k in range(N):
        tosum.append(inner_sum(k, j, i_range))
    return sum(tosum)

In [39]:
# do the 1/n and the sqrt
def eqn22(N, j, i_range):
    return (1/N) * np.sqrt(outer_sum(N, j, i_range))

In [40]:
# do the sensitivity analysis
i_range = len(x0)
j_range = len(ks.keys())
OS = []
for j in range(j_range):
    print('Calculating sensitivity for j {}/{}\r'.format(j + 1, j_range), end='')
    OS.append(eqn22(N, j, i_range))
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
print(OS)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [23]:
np.argmax(OS)

20

In [24]:
counted = [(i, OS[i]) for i in range(len(OS))]

In [25]:
counted.sort(key=lambda x: x[1], reverse=True)

In [26]:
print(counted)

[(20, 1619.8828868544567), (12, 330.9632090035078), (6, 180.74746402541157), (4, 137.31842466976738), (18, 37.24247375547597), (22, 11.377239632599947), (2, 1.0499135485109066), (9, 0.6458599759069408), (0, 0.1688541025122092), (24, 0.11966240132508452), (51, 0.05275313452106152), (57, 0.023340834113016518), (54, 0.01804261951158499), (45, 0.014093320915467203), (39, 0.013394138420144735), (47, 0.012525162462513348), (41, 0.01224817427317476), (35, 0.01172759050513242), (11, 0.011541114672534672), (29, 0.006805313607960929), (31, 0.006805311568527983), (8, 0.006746562135238839), (14, 0.006236568747714927), (33, 0.00519321702566376), (27, 0.003902477700030688), (53, 0.003253936206770214), (1, 0.00209460485655392), (59, 0.0012197232504302224), (56, 0.0012194936619451965), (15, 0.001137061149792114), (13, 0.001070647750258728), (61, 0.0009625615779393127), (63, 0.0005203035556428718), (40, 0.00041387782835228455), (46, 0.00041387782835228455), (55, 0.00041387782835228455), (58, 0.00041387